## Download Data from Google-Drive

In [ ]:
'''
File Name : Data
File Link : https://drive.google.com/file/d/1rWS8Jj19ZOzdXFR4jUjmHIx_fTRyYzOw/view?usp=share_link
File Id : '1rWS8Jj19ZOzdXFR4jUjmHIx_fTRyYzOw'

'''
!gdown --id 1rWS8Jj19ZOzdXFR4jUjmHIx_fTRyYzOw
!unzip ccle_ctrpv2_gdse.zip

## Import necessary libraries

In [11]:
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.callbacks import ModelCheckpoint
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import KFold
from scipy.stats import pearsonr
import matplotlib.pyplot as plt
from sklearn.metrics import mean_squared_error , mean_absolute_error
from time import perf_counter, sleep
import math
import scipy.stats as stats
import warnings
from sklearn.model_selection import train_test_split
warnings.filterwarnings('ignore')

## Read train and test data

In [12]:
# x_train
ccle_ctrpv2 = pd.read_csv("../data/ccle_ctrpv2_rnaseq_tpm.csv")
# x_test
ccle_ctrpv2_aac = pd.read_csv("../data/ccle_ctrpv2_aac.csv")
# y_train
gdse_rnaseq = pd.read_csv("../data/gdse_rnaseq_tpm.csv")
# y_test
gdse_aac = pd.read_csv("../data/gdse_aac.csv")

## Cleaning the dataframes

In [13]:
drug_names = gdse_aac.columns.values.tolist()
drug_names = drug_names[1:]


ccle_ctrpv2 = ccle_ctrpv2.rename(columns={"Unnamed: 0": "sample"})
gdse_rnaseq = gdse_rnaseq.rename(columns={"Unnamed: 0": "sample"})
ccle_ctrpv2_aac = ccle_ctrpv2_aac.rename(columns={"Unnamed: 0": "sample"})
gdse_aac = gdse_aac.rename(columns={"Unnamed: 0": "sample"})

In [14]:
def remove_nan(data):
    data = data.dropna()
    data.reset_index(drop = True)
    return data

In [15]:
'''
Initializing k-fold cross-validation.

n_splits=4: the dataset will be split into 4 equal-sized folds or subsets.

shuffle: data will be shuffled before splitting into folds, 
         ensuring randomization in the distribution of samples across each fold.

'''
kfold = KFold(n_splits=4, shuffle=True)


'''
Initializing StandardScaler to normalize the data to have mean of 0 and variance of 1.
'''
Sscaler = StandardScaler()

## Creating a shallow neural network
---
1. **Layers**

First Hidden Layer -> **Activation Function:** *Rectified Linear Unit (Relu)* <br>
Second Hidden Layer -> **Activation Function:** *Rectified Linear Unit (Relu)* <br>
Third Hidden Layer -> **Activation Function:** *Rectified Linear Unit (Relu)* <br>
Fourth Hidden Layer -> **Activation Function:** *Rectified Linear Unit (Relu)* <br>

Output Layer -> **Activation Function:** *Sigmoid* <br>

Sigmoid activation function is used to force the output to have a range between 0 and 1, 
as the desired output (Drug Response) has a value between 0 and 1.

2. **Optimizer**

Using *Stochastic Gradient descent* (SGD) with momentum:

        learning rate: 0.01
        momentum = 0.9

3. **Loss Function**

Using *mean squared error* (MSE) for the loss function

4. **Regularization technique**

20% dropout after each hidden layer 

5. **Network schematic**

![MarineGEO circle logo](Arch.png)

In [16]:
model = tf.keras.models.Sequential()
model.add(Dense(256, activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(1 , activation='sigmoid'))



learning_rate = 0.01
momentum = 0.9
optimizer_sgd = tf.keras.optimizers.SGD(learning_rate, momentum)

optimizer_rmsprop = tf.keras.optimizers.RMSprop()

In [17]:
# setting the number of features to be used based on variance:
num_features_to_select = 100

# Training procedure

*For each drug in the drug list:*
   1. Split the data in to train and test sets, and then remove the Nan values.
   2. Merging X_train with ytrain, and X_test with  y_test because the samples in each are different.
   3. Normalize the training X. </br>
   
   *Running the following 6 times:*
   
       4. Applying 4-fold Cross Valisation
       5. Applying the relevant features selection technique, and extract the selected features.
       6. Train the model based on the CV data and extracted features
       7. Saving the best model performed in CV and the best features
       
   8. Predicting the test set using the best model from the previous section
   9. Calculating the correlation score between the predicted drug_respons and the actual drug_response

In [18]:
cor_df = pd.DataFrame([], columns = ['corr'])

correlatoins = []
correlatoins = []
mse_train_list = []
rmse_train_list = []
mae_train_list = []
spearmanr_correlation_train_list = []
kendalltau_correlation_train_list = []
cr = []
mse = []
rmse = []
mae = []
spearmanr =[]
kendalltau = []
start = perf_counter()
for drug in drug_names:
    
    selected_features = []
    best_model_top_features = set()
    model_performances = []
    model_acc = []
    pearson_correlations = []
    saved_features = []


    # train test split

    y_train = ccle_ctrpv2_aac[['sample', drug]]
    y_test  = gdse_aac[['sample', drug]]

    y_train = remove_nan(y_train)
    y_test  = remove_nan(y_test)

    ccle_ctrpv2 = remove_nan(ccle_ctrpv2)
    gdse_rnaseq = remove_nan(gdse_rnaseq)


    # Merge

    gdse_rnaseq_merged_df = pd.merge(gdse_rnaseq, y_test, on='sample', how='inner')
    ccle_ctrpv2_merged_df = pd.merge(ccle_ctrpv2, y_train, on='sample', how='inner')

    X_train = ccle_ctrpv2_merged_df.iloc[:, 1: len(gdse_rnaseq.columns)]
    X_test  = gdse_rnaseq_merged_df.iloc[:, 1: len(gdse_rnaseq.columns)]

    Y_train = ccle_ctrpv2_merged_df.iloc[:, len(gdse_rnaseq.columns):]
    Y_test  = gdse_rnaseq_merged_df.iloc[:, len(gdse_rnaseq.columns):]
    
    # Normalize the Xtrain

    X_train_norm = Sscaler.fit_transform(X_train)
    X_train_normalized = pd.DataFrame(X_train_norm, columns=X_train.columns)
    
    for i in range(6):
            
        selected_features = []
        best_model_top_features = set()
        model_performances = []
        model_acc = []
        pearson_correlations = []
        best_model_path = 'saved_models/best_model_'+ drug + str(i)
        saved_features = []
        
        for train, validation in kfold.split(X_train_normalized , Y_train):

            train_data = X_train_normalized.iloc[train]
            train_labels = Y_train.iloc[train]
            val_data = X_train_normalized.iloc[validation]
            val_labels = Y_train.iloc[validation]


            feature_variances = np.var(train_data, axis=0)
            top_features_indices = feature_variances.sort_values()[-num_features_to_select:]
            top_features = np.asarray(top_features_indices.index)


            train_data_selected = train_data[top_features]
            val_data_selected = val_data[top_features]


            model.compile(optimizer= optimizer_sgd, loss='mean_squared_error', metrics='MAE')

            checkpoint_callback = ModelCheckpoint(filepath=best_model_path+ '.h5' , monitor='MAE', save_best_only=True, mode='min')

            history = model.fit(train_data_selected, train_labels, epochs=15,callbacks=[checkpoint_callback], validation_data=(val_data_selected, val_labels))
            saved_features.append({
                'top_features': top_features,
                'model_performances': history.history['MAE'][-1]
            })

        saved_features = sorted(saved_features, key=lambda x: x['model_performances'])
        top_features_saved = saved_features[0].get('top_features')
        top_features_saved = np.asarray(top_features_saved)
        new_X = X_test[top_features_saved]

        Xtest_norm = Sscaler.fit_transform(new_X)
        Xtest_normalized = pd.DataFrame(Xtest_norm, columns=new_X.columns)

        loaded_model = tf.keras.models.load_model(best_model_path+ '.h5' )

        gdsc_predictions = loaded_model.predict(Xtest_normalized)

        Y_test_array = np.asarray(Y_test) 

        correlation, _ = pearsonr(Y_test_array.flatten(),  gdsc_predictions.flatten())
        mse_train = mean_squared_error(Y_test_array.flatten(), gdsc_predictions.flatten())
        rmse_train = math.sqrt(mse_train)
        mae_train = mean_absolute_error(Y_test_array.flatten(), gdsc_predictions.flatten())

    # Calculate Correlation for the validation data
        spearmanr_correlation_train, _ = stats.spearmanr(Y_test_array.flatten(), gdsc_predictions.flatten())
        kendalltau_correlation_train, _ = stats.kendalltau(Y_test_array.flatten(), gdsc_predictions.flatten())
    
        correlatoins.append(correlation)
        mse_train_list.append(mse_train)
        rmse_train_list.append(rmse_train)
        mae_train_list.append(mae_train)
        spearmanr_correlation_train_list.append(spearmanr_correlation_train)
        kendalltau_correlation_train_list.append(kendalltau_correlation_train)
        
        
        
    index = correlatoins.index(max(correlatoins))
    cr.append(max(correlatoins))
    mse.append(mse_train_list[index])
    rmse.append(rmse_train_list[index])
    mae.append(mae_train_list[index])
    spearmanr.append(spearmanr_correlation_train_list[index])
    kendalltau.append(kendalltau_correlation_train_list[index])
    correlatoins = []
    mse_train_list = []
    rmse_train_list = []
    mae_train_list = []
    spearmanr_correlation_train_list = []
    kendalltau_correlation_train_list = []
    
    
    
end = perf_counter()

Epoch 1/15
18/18 [==============================] - 1s 18ms/step - loss: 0.0835 - MAE: 0.2537 - val_loss: 0.0280 - val_MAE: 0.1461
Epoch 2/15
18/18 [==============================] - 0s 10ms/step - loss: 0.0184 - MAE: 0.1106 - val_loss: 0.0121 - val_MAE: 0.0886
Epoch 3/15
18/18 [==============================] - 0s 9ms/step - loss: 0.0172 - MAE: 0.1061 - val_loss: 0.0110 - val_MAE: 0.0846
Epoch 4/15
18/18 [==============================] - 0s 9ms/step - loss: 0.0157 - MAE: 0.1024 - val_loss: 0.0116 - val_MAE: 0.0870
Epoch 5/15
18/18 [==============================] - 0s 9ms/step - loss: 0.0158 - MAE: 0.1015 - val_loss: 0.0119 - val_MAE: 0.0885
Epoch 6/15
18/18 [==============================] - 0s 9ms/step - loss: 0.0155 - MAE: 0.1003 - val_loss: 0.0119 - val_MAE: 0.0891
Epoch 7/15
18/18 [==============================] - 0s 9ms/step - loss: 0.0141 - MAE: 0.0953 - val_loss: 0.0114 - val_MAE: 0.0871
Epoch 8/15
18/18 [==============================] - 0s 9ms/step - loss: 0.0131 - MAE: 0.

18/18 [==============================] - 0s 8ms/step - loss: 0.0086 - MAE: 0.0749 - val_loss: 0.0076 - val_MAE: 0.0717
Epoch 4/15
18/18 [==============================] - 0s 8ms/step - loss: 0.0086 - MAE: 0.0747 - val_loss: 0.0075 - val_MAE: 0.0711
Epoch 5/15
18/18 [==============================] - 0s 9ms/step - loss: 0.0080 - MAE: 0.0718 - val_loss: 0.0074 - val_MAE: 0.0705
Epoch 6/15
18/18 [==============================] - 0s 7ms/step - loss: 0.0083 - MAE: 0.0733 - val_loss: 0.0073 - val_MAE: 0.0700
Epoch 7/15
18/18 [==============================] - 0s 7ms/step - loss: 0.0082 - MAE: 0.0720 - val_loss: 0.0072 - val_MAE: 0.0694
Epoch 8/15
18/18 [==============================] - 0s 9ms/step - loss: 0.0077 - MAE: 0.0701 - val_loss: 0.0071 - val_MAE: 0.0688
Epoch 9/15
18/18 [==============================] - 0s 7ms/step - loss: 0.0076 - MAE: 0.0708 - val_loss: 0.0070 - val_MAE: 0.0683
Epoch 10/15
18/18 [==============================] - 0s 9ms/step - loss: 0.0076 - MAE: 0.0689 - val_l

18/18 [==============================] - 0s 7ms/step - loss: 0.0078 - MAE: 0.0702 - val_loss: 0.0053 - val_MAE: 0.0596
Epoch 6/15
18/18 [==============================] - 0s 7ms/step - loss: 0.0079 - MAE: 0.0705 - val_loss: 0.0052 - val_MAE: 0.0595
Epoch 7/15
18/18 [==============================] - 0s 9ms/step - loss: 0.0077 - MAE: 0.0689 - val_loss: 0.0052 - val_MAE: 0.0594
Epoch 8/15
18/18 [==============================] - 0s 7ms/step - loss: 0.0077 - MAE: 0.0692 - val_loss: 0.0052 - val_MAE: 0.0592
Epoch 9/15
18/18 [==============================] - 0s 7ms/step - loss: 0.0076 - MAE: 0.0691 - val_loss: 0.0052 - val_MAE: 0.0591
Epoch 10/15
18/18 [==============================] - 0s 7ms/step - loss: 0.0076 - MAE: 0.0689 - val_loss: 0.0051 - val_MAE: 0.0589
Epoch 11/15
18/18 [==============================] - 0s 9ms/step - loss: 0.0074 - MAE: 0.0685 - val_loss: 0.0051 - val_MAE: 0.0588
Epoch 12/15
18/18 [==============================] - 0s 9ms/step - loss: 0.0073 - MAE: 0.0678 - val

18/18 [==============================] - 0s 9ms/step - loss: 0.0067 - MAE: 0.0660 - val_loss: 0.0068 - val_MAE: 0.0627
Epoch 8/15
18/18 [==============================] - 0s 10ms/step - loss: 0.0066 - MAE: 0.0647 - val_loss: 0.0068 - val_MAE: 0.0627
Epoch 9/15
18/18 [==============================] - 0s 8ms/step - loss: 0.0072 - MAE: 0.0676 - val_loss: 0.0068 - val_MAE: 0.0626
Epoch 10/15
18/18 [==============================] - 0s 8ms/step - loss: 0.0069 - MAE: 0.0659 - val_loss: 0.0068 - val_MAE: 0.0626
Epoch 11/15
18/18 [==============================] - 0s 8ms/step - loss: 0.0067 - MAE: 0.0654 - val_loss: 0.0068 - val_MAE: 0.0626
Epoch 12/15
18/18 [==============================] - 0s 8ms/step - loss: 0.0069 - MAE: 0.0670 - val_loss: 0.0068 - val_MAE: 0.0626
Epoch 13/15
18/18 [==============================] - 0s 8ms/step - loss: 0.0067 - MAE: 0.0659 - val_loss: 0.0067 - val_MAE: 0.0627
Epoch 14/15
18/18 [==============================] - 0s 9ms/step - loss: 0.0066 - MAE: 0.0655 - 

18/18 [==============================] - 0s 9ms/step - loss: 0.0068 - MAE: 0.0649 - val_loss: 0.0061 - val_MAE: 0.0615
Epoch 10/15
18/18 [==============================] - 0s 7ms/step - loss: 0.0068 - MAE: 0.0653 - val_loss: 0.0061 - val_MAE: 0.0615
Epoch 11/15
18/18 [==============================] - 0s 8ms/step - loss: 0.0074 - MAE: 0.0674 - val_loss: 0.0061 - val_MAE: 0.0615
Epoch 12/15
18/18 [==============================] - 0s 7ms/step - loss: 0.0070 - MAE: 0.0665 - val_loss: 0.0061 - val_MAE: 0.0615
Epoch 13/15
18/18 [==============================] - 0s 8ms/step - loss: 0.0071 - MAE: 0.0664 - val_loss: 0.0061 - val_MAE: 0.0615
Epoch 14/15
18/18 [==============================] - 0s 8ms/step - loss: 0.0068 - MAE: 0.0656 - val_loss: 0.0061 - val_MAE: 0.0615
Epoch 15/15
18/18 [==============================] - 0s 8ms/step - loss: 0.0067 - MAE: 0.0650 - val_loss: 0.0061 - val_MAE: 0.0615
Epoch 1/15
18/18 [==============================] - 1s 19ms/step - loss: 0.0070 - MAE: 0.0659 -

...

17/17 [==============================] - 0s 19ms/step - loss: 0.0131 - MAE: 0.0921 - val_loss: 0.0155 - val_MAE: 0.0977
Epoch 1/15
17/17 [==============================] - 1s 30ms/step - loss: 0.0143 - MAE: 0.0960 - val_loss: 0.0120 - val_MAE: 0.0866
Epoch 2/15
17/17 [==============================] - 0s 17ms/step - loss: 0.0143 - MAE: 0.0960 - val_loss: 0.0120 - val_MAE: 0.0867
Epoch 3/15
17/17 [==============================] - 0s 17ms/step - loss: 0.0143 - MAE: 0.0964 - val_loss: 0.0120 - val_MAE: 0.0868
Epoch 4/15
17/17 [==============================] - 0s 17ms/step - loss: 0.0143 - MAE: 0.0964 - val_loss: 0.0120 - val_MAE: 0.0869
Epoch 5/15
17/17 [==============================] - 0s 16ms/step - loss: 0.0143 - MAE: 0.0964 - val_loss: 0.0120 - val_MAE: 0.0869
Epoch 6/15
17/17 [==============================] - 0s 16ms/step - loss: 0.0144 - MAE: 0.0966 - val_loss: 0.0121 - val_MAE: 0.0869
Epoch 7/15
17/17 [==============================] - 0s 17ms/step - loss: 0.0143 - MAE: 0.0965 

17/17 [==============================] - 0s 19ms/step - loss: 0.0129 - MAE: 0.0908 - val_loss: 0.0161 - val_MAE: 0.1023
Epoch 3/15
17/17 [==============================] - 0s 17ms/step - loss: 0.0129 - MAE: 0.0908 - val_loss: 0.0161 - val_MAE: 0.1024
Epoch 4/15
17/17 [==============================] - 0s 19ms/step - loss: 0.0129 - MAE: 0.0905 - val_loss: 0.0161 - val_MAE: 0.1024
Epoch 5/15
17/17 [==============================] - 0s 17ms/step - loss: 0.0128 - MAE: 0.0906 - val_loss: 0.0161 - val_MAE: 0.1025
Epoch 6/15
17/17 [==============================] - 0s 19ms/step - loss: 0.0128 - MAE: 0.0904 - val_loss: 0.0160 - val_MAE: 0.1026
Epoch 7/15
17/17 [==============================] - 0s 16ms/step - loss: 0.0129 - MAE: 0.0906 - val_loss: 0.0160 - val_MAE: 0.1025
Epoch 8/15
17/17 [==============================] - 0s 17ms/step - loss: 0.0129 - MAE: 0.0907 - val_loss: 0.0160 - val_MAE: 0.1025
Epoch 9/15
17/17 [==============================] - 0s 19ms/step - loss: 0.0128 - MAE: 0.0903 

17/17 [==============================] - 0s 17ms/step - loss: 0.0140 - MAE: 0.0947 - val_loss: 0.0127 - val_MAE: 0.0903
Epoch 5/15
17/17 [==============================] - 0s 17ms/step - loss: 0.0139 - MAE: 0.0946 - val_loss: 0.0127 - val_MAE: 0.0903
Epoch 6/15
17/17 [==============================] - 0s 17ms/step - loss: 0.0139 - MAE: 0.0947 - val_loss: 0.0127 - val_MAE: 0.0903
Epoch 7/15
17/17 [==============================] - 0s 19ms/step - loss: 0.0138 - MAE: 0.0940 - val_loss: 0.0127 - val_MAE: 0.0904
Epoch 8/15
17/17 [==============================] - 0s 17ms/step - loss: 0.0138 - MAE: 0.0942 - val_loss: 0.0127 - val_MAE: 0.0904
Epoch 9/15
17/17 [==============================] - 0s 16ms/step - loss: 0.0138 - MAE: 0.0943 - val_loss: 0.0127 - val_MAE: 0.0904
Epoch 10/15
17/17 [==============================] - 0s 17ms/step - loss: 0.0137 - MAE: 0.0941 - val_loss: 0.0127 - val_MAE: 0.0904
Epoch 11/15
17/17 [==============================] - 0s 19ms/step - loss: 0.0136 - MAE: 0.093

17/17 [==============================] - 0s 19ms/step - loss: 0.0140 - MAE: 0.0944 - val_loss: 0.0128 - val_MAE: 0.0913
Epoch 7/15
17/17 [==============================] - 0s 19ms/step - loss: 0.0140 - MAE: 0.0939 - val_loss: 0.0128 - val_MAE: 0.0913
Epoch 8/15
17/17 [==============================] - 0s 16ms/step - loss: 0.0140 - MAE: 0.0943 - val_loss: 0.0128 - val_MAE: 0.0913
Epoch 9/15
17/17 [==============================] - 0s 17ms/step - loss: 0.0140 - MAE: 0.0942 - val_loss: 0.0128 - val_MAE: 0.0912
Epoch 10/15
17/17 [==============================] - 0s 17ms/step - loss: 0.0140 - MAE: 0.0944 - val_loss: 0.0128 - val_MAE: 0.0912
Epoch 11/15
17/17 [==============================] - 0s 17ms/step - loss: 0.0140 - MAE: 0.0943 - val_loss: 0.0128 - val_MAE: 0.0912
Epoch 12/15
17/17 [==============================] - 0s 17ms/step - loss: 0.0140 - MAE: 0.0942 - val_loss: 0.0128 - val_MAE: 0.0912
Epoch 13/15
17/17 [==============================] - 0s 17ms/step - loss: 0.0140 - MAE: 0.0

17/17 [==============================] - 0s 19ms/step - loss: 0.0135 - MAE: 0.0924 - val_loss: 0.0127 - val_MAE: 0.0905
Epoch 9/15
17/17 [==============================] - 0s 20ms/step - loss: 0.0135 - MAE: 0.0924 - val_loss: 0.0126 - val_MAE: 0.0905
Epoch 10/15
17/17 [==============================] - 0s 19ms/step - loss: 0.0135 - MAE: 0.0925 - val_loss: 0.0126 - val_MAE: 0.0904
Epoch 11/15
17/17 [==============================] - 0s 17ms/step - loss: 0.0135 - MAE: 0.0928 - val_loss: 0.0126 - val_MAE: 0.0904
Epoch 12/15
17/17 [==============================] - 0s 18ms/step - loss: 0.0134 - MAE: 0.0932 - val_loss: 0.0126 - val_MAE: 0.0903
Epoch 13/15
17/17 [==============================] - 0s 24ms/step - loss: 0.0134 - MAE: 0.0923 - val_loss: 0.0126 - val_MAE: 0.0903
Epoch 14/15
17/17 [==============================] - 0s 23ms/step - loss: 0.0133 - MAE: 0.0920 - val_loss: 0.0126 - val_MAE: 0.0902
Epoch 15/15
17/17 [==============================] - 0s 19ms/step - loss: 0.0133 - MAE: 0

# Saving the Results

In [19]:
execution_time = end - start

df = pd.DataFrame({"Drugs": drug_names, "pearsonCor": cr , "spearmanCor": spearmanr , "kendallCor": kendalltau , 
                  "RMSE": rmse ,  "MSE": mse , "MAE": mae , "Time": execution_time})
df.to_csv('DL-results/Result_var100.csv', index=False)